# [Sql Server First Reponder Kit](https://github.com/BrentOzarULTD/SQL-Server-First-Responder-Kit/tree/main)

You're a DBA, sysadmin, or developer who manages Microsoft SQL Servers. It's your fault if they're down or slow. These tools help you understand what's going on in your server.

- When you want an overall health check, run sp\_Blitz.
- To learn which queries have been using the most resources, run sp\_BlitzCache.
- To analyze which indexes are missing or slowing you down, run sp\_BlitzIndex.
- To find out why the server is slow right now, run sp\_BlitzFirst.

## Install First Responder Kit

In [ ]:
Import-Module dbatools
$ServerName = @('localhost')

Install-DbaFirstResponderKit -SqlInstance $ServerName -Database master -Verbose

## [sp\_Blitz: Overall Health Check](https://github.com/BrentOzarULTD/SQL-Server-First-Responder-Kit/tree/main#sp_blitz-overall-health-check)

In [5]:
$sp_BlitzResult = Invoke-DbaQuery -SqlInstance $ServerName -Database master -Query "EXEC sp_blitz @CheckServerInfo = 1"
$sp_BlitzResult | Out-GridView

## [Logging sp\_BlitzFirst using SQL Agent Job](https://github.com/BrentOzarULTD/SQL-Server-First-Responder-Kit/tree/main#logging-sp_blitzfirst-to-tables)  

Execute below tsql that creates job named '' with schedule of every 15 minutes.

Make sure a Job Category named '<span style="color: rgb(163, 21, 21); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">(dba)&nbsp;Monitoring&nbsp;&amp;&nbsp;Alerting</span>' exists. Also, chage value of <span style="color: rgb(163, 21, 21); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">@OutputDatabaseName</span> on **line 39** if required.

In [7]:
$sqlCreateJob = @"
USE [msdb]
GO

BEGIN TRANSACTION
DECLARE @ReturnCode INT
SELECT @ReturnCode = 0
IF NOT EXISTS (SELECT name FROM msdb.dbo.syscategories WHERE name=N'(dba) Monitoring & Alerting' AND category_class=1)
BEGIN
EXEC @ReturnCode = msdb.dbo.sp_add_category @class=N'JOB', @type=N'LOCAL', @name=N'(dba) Monitoring & Alerting'
IF (@@ERROR <> 0 OR @ReturnCode <> 0) GOTO QuitWithRollback

END

DECLARE @jobId BINARY(16)
EXEC @ReturnCode =  msdb.dbo.sp_add_job @job_name=N'(dba) Run First-Responder-Kit', 
		@enabled=1, 
		@notify_level_eventlog=0, 
		@notify_level_email=0, 
		@notify_level_netsend=0, 
		@notify_level_page=0, 
		@delete_level=0, 
		@description=N'Caputure stats using sp_BlitzFirst', 
		@category_name=N'(dba) Monitoring & Alerting', 
		@owner_login_name=N'sa', @job_id = @jobId OUTPUT
IF (@@ERROR <> 0 OR @ReturnCode <> 0) GOTO QuitWithRollback

EXEC @ReturnCode = msdb.dbo.sp_add_jobstep @job_id=@jobId, @step_name=N'(dba) Run First-Responder-Kit', 
		@step_id=1, 
		@cmdexec_success_code=0, 
		@on_success_action=1, 
		@on_success_step_id=0, 
		@on_fail_action=2, 
		@on_fail_step_id=0, 
		@retry_attempts=0, 
		@retry_interval=0, 
		@os_run_priority=0, @subsystem=N'TSQL', 
		@command=N'exec sp_BlitzFirst
	@OutputDatabaseName = ''DBA'',
	@OutputSchemaName = ''dbo'',
	@OutputTableName = ''BlitzFirst'',
	@OutputTableNameFileStats = ''BlitzFirst_FileStats'',
	@OutputTableNamePerfmonStats = ''BlitzFirst_PerfmonStats'',
	@OutputTableNameWaitStats = ''BlitzFirst_WaitStats'',
	@OutputTableNameBlitzCache = ''BlitzCache'',
	@OutputTableNameBlitzWho = ''BlitzWho''', 
		@database_name=N'master', 
		@flags=12
IF (@@ERROR <> 0 OR @ReturnCode <> 0) GOTO QuitWithRollback
EXEC @ReturnCode = msdb.dbo.sp_update_job @job_id = @jobId, @start_step_id = 1
IF (@@ERROR <> 0 OR @ReturnCode <> 0) GOTO QuitWithRollback
EXEC @ReturnCode = msdb.dbo.sp_add_jobschedule @job_id=@jobId, @name=N'(dba) Run First-Responder-Kit', 
		@enabled=1, 
		@freq_type=4, 
		@freq_interval=1, 
		@freq_subday_type=4, 
		@freq_subday_interval=10, 
		@freq_relative_interval=0, 
		@freq_recurrence_factor=0, 
		@active_start_date=20220318, 
		@active_end_date=99991231, 
		@active_start_time=0, 
		@active_end_time=235959, 
		@schedule_uid=N'fcd79fa8-3d9d-4b30-88d0-9a14081e3339'
IF (@@ERROR <> 0 OR @ReturnCode <> 0) GOTO QuitWithRollback
EXEC @ReturnCode = msdb.dbo.sp_add_jobserver @job_id = @jobId, @server_name = N'(local)'
IF (@@ERROR <> 0 OR @ReturnCode <> 0) GOTO QuitWithRollback
COMMIT TRANSACTION
GOTO EndSave
QuitWithRollback:
    IF (@@TRANCOUNT > 0) ROLLBACK TRANSACTION
EndSave:
GO

"@

Invoke-DbaQuery -SqlInstance $ServerName -Database master -Query $sqlCreateJob

$sqlStartJob = "exec msdb..sp_start_job @job_name = '(dba) Run First-Responder-Kit'"
Invoke-DbaQuery -SqlInstance $ServerName -Database master -Query $sqlStartJob
